Предположим мы хотим составить датасет с рок-концертами в крупных городах в России за последние 20 лет. Например, чтобы выяснить, какая группа выступала чаще.

Скорее всего, через API мы такую информацию не вытащим (хотя бы потому, что там точно нет данных до 2007 года), зато она есть на профильных сайтах, например, на https://darkside.ru/show/. 
Для начала научимся скачивать html код страницы.

In [ ]:
import requests

answer = requests.get("https://darkside.ru/show/")
print(answer.text[:2000])
print("... и так далее")

Чтобы понять, правда ли мы скачали страницу с данными, а не страницу с ошибкой, можно посмотреть на код ответа. Если все хорошо - код ответа 200 (а если страница не найдена - 404).

In [18]:
print(answer.status_code)

200


Для того, чтобы "вытягивать" нужную нам информацию из html кода, можно использовать библиотеку beautiful soup, она умеет строить дерево из тегов и производить поиск в нем.

In [35]:
!pip install beautifulsoup4

In [36]:
from bs4 import BeautifulSoup

document = BeautifulSoup(answer.text, 'html.parser')

Попробуем для начала найти все ссылки и выведем их на экран.

In [58]:
links = document.find_all('a')
for link in links[:20]:
    print(link, end='\n----\n')

<a name="top"><table border="0" cellpadding="0" cellspacing="0" height="162" width="100%"><tr valign="BOTTOM"><td align="center" background="/img/top-1-bg.png" height="24"><img height="1" src="/img/blank.gif" style="visibility:hidden" width="45"/></td><td align="center" background="/img/top-1-bg.png" height="24"><center><img height="24" src="/img/blank.gif" style="visibility:hidden" width="1"/><span id="menulayer0" onmouseout="initiateHideMenu(0); return true" onmouseover="showMenu(0); return true"><img border="0" height="24" src="/img/top-1-2-events.png" width="72"/></span><img border="0" height="24" src="/img/top-1-3.png" width="29"/><span id="menulayer1" onmouseout="initiateHideMenu(1); return true" onmouseover="showMenu(1); return true"><img border="0" height="24" src="/img/top-1-4-music.png" width="72"/></span><img border="0" height="24" src="/img/top-1-5.png" width="10"/><img border="0" height="24" src="/img/top-1-logo.jpg" width="427"/><img border="0" height="24" src="/img/top-1

Как мы видим, среди ссылок есть нужные, но есть и много мусора. Попробуем отфильтровать ненужное: как минимум убрать ссылки с пустым текстом или без href.

In [59]:
links = document.find_all('a')
for link in links[:20]:
    if link.text == '':
        continue
    if 'href' not in link.attrs:
        continue
    print(link, end='\n----\n')

<a href="/show/index.phtml?cp=0"><font color="#940D0D">1</font></a>
----
<a href="/show/index.phtml?cp=9405"><font color="#940D0D">100</font></a>
----
<a href="/show/index.phtml?cp=9500"><font color="#940D0D">101</font></a>
----
<a href="<img alt="Рейтинг@" height="31" src="" style = "border:0;" width="88"/></a>
----
<a href=" color="#650D0A"> 1997-2019 © Russian Darkside e-Zine.</font></a>
----
<a href="/feedback/">сообщите нам об этом</a>
----
<a href="<img alt="" border="0" height="1" src="" width = "1"/></a>
----
<a href="/news/" onmouseout=".color='#C08A56';return true" onmouseover=".color='#F0A754';return true" style="color:#C08A56">Новости</a>
----


Кажется, мы отобрали нужные строки. Теперь попробуем понять, что есть что. Например, на основе href.

In [60]:
for link in document.find_all('a'):
    if link.text == '':
        continue
    if 'href' not in link.attrs:
        continue
    if '/show/' not in link.attrs.get('href') or  '/show/' == link.attrs.get('href'):
        continue
    if '/show/index.phtml' in link.attrs.get('href'):
        continue
    if "club" in link.attrs.get('href'):
        print("Клуб", link.text)
    elif "town" in link.attrs.get('href'):
        print("Город", link.text)
    elif "promoter" in link.attrs.get('href'):
        print("Организатор", link.text)
    else:
        print('---' * 3)
        print("Группа", link.text)

Осталось сложить это все в датасет, скачав данные со всех страниц.

In [63]:
import pandas as pd

concerts = pd.DataFrame(columns=['band', 'town', 'club', 'promoter'])
i = 0
concert = {'club': '', 'promoter': '', 'band': '', 'town': ''}

for p in range(0, 90):
    print(".", end='')
    answer = requests.get(f' * 108}')
    document = BeautifulSoup(answer.text, 'html.parser')

    for link in document.find_all('a'):
        if link.text == '':
            continue
        if 'href' not in link.attrs:
            continue
        if '/show/' not in link.attrs.get('href') or  '/show/' == link.attrs.get('href'):
            continue
        if '/show/index.phtml' in link.attrs.get('href'):
            continue
        if "club" in link.attrs.get('href'):
            concert['club'] = link.text
        elif "town" in link.attrs.get('href'):
            concert['town'] = link.text
        elif "promoter" in link.attrs.get('href'):
            concert['promoter'] = link.text
        else:
            if concert['band'] != '':
                concerts.loc[i] = (concert['band'], concert['town'], concert['club'], concert['promoter'])
                i += 1
            concert = {'club': '', 'promoter': '', 'band': '', 'town': ''}
            concert['band'] = link.text
    concerts.loc[i]  = (concert['band'], concert['town'], concert['club'], concert['promoter'])
concerts

..........................................................................................

,band,town,club,promoter
0,Hatari,Москва,Клуб Театръ,Russian Synth Community
1,Guano Apes,Санкт-Петербург,Aurora Hall,Delta Mekong Concert
2,Saor,Москва,Pravda,Ritual Booking
3,Theory,Москва,1930,Мельница
4,Чёрный Обелиск,Москва,Red Club,
5,Guano Apes,Москва,Adrenaline Stadium,JC Sound
6,Motionless In White,Москва,ГлавClub GreenConcert (ex-YotaSpace),SPIKA Concert Agency
7,Motionless In White,Санкт-Петербург,Zal,SPIKA Concert Agency
8,Weedeater,Санкт-Петербург,MOD,MadStream Booking
9,1,,,


In [64]:
print(len(concerts))

10335


Теперь можем даже посчитать, кто выступал чаще. А еще встречаются странные числа...

Данные выше в приципе можно дополнительно почитстить, например, разделить группы на фестивалях, убрать слово концерт.

In [65]:
pd.value_counts(concerts['band'])[:30]

1                  648
2                  300
3                  184
4                  141
5                   94
6                   76
Эпидемия            55
7                   55
8                   47
9                   42
10                  39
13                  29
Мельница            29
11                  27
Ария                27
Lacrimosa           25
12                  25
14                  23
U.D.O.              21
Papa Roach          20
Scorpions           19
Kreator             18
Кипелов             18
Accept              17
Тролль Гнет Ель     17
W.A.S.P.            17
15                  16
Tarja Turunen       16
16                  16
Anathema            15
Name: band, dtype: int64